<a href="https://colab.research.google.com/github/gokcergun/road_sign_detection/blob/main/Road_Sign_MaskRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For MaskRCNN we will use MMDetection implmentation.
<a href="https://github.com/open-mmlab/mmdetection">MMDetection</a> is an open source object detection toolbox based on PyTorch. It is a part of <a href="https://openmmlab.com">OpenMMLab</a> consists of more than 100 pre-trained models to use for fine-tuning for object detection and instance segmentation tasks. 

# 1. Set-up


### Connect GoogleDrive
Data_path: "/content/drive/MyDrive/Datasets/data_sign_coco"

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Install MMDetection

In [2]:
# Check GPU setting of GoogleColabs
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 704.4 MB 1.3 kB/s 
     |████████████████████████████████| 6.6 MB 1.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
     |████████████████████████████████| 416 kB 4.1 MB/s 
     |██████████████████

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 False
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
2.20.0
not available
GCC 7.5


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 False
2.20.0
not available
GCC 7.5


## Data Format
MMDetection library requires the annotations as json files in COCO format with the following necessary keys:
```
'images': [
    {
        'file_name': 'COCO_val2014_000000001268.jpg',
        'height': 427,
        'width': 640,
        'id': 1268
    },
    ...
],

'annotations': [
    {
        'segmentation': [[192.81,
            247.09,
            ...
            219.03,
            249.06]],  # if you have mask labels
        'area': 1035.749,
        'iscrowd': 0,
        'image_id': 1268,
        'bbox': [192.81, 224.8, 74.73, 33.43],
        'category_id': 16,
        'id': 42986
    },
    ...
],

'categories': [
    {'id': 0, 'name': 'car'},
 ]
```




## RoadSignDataset  
We first converted our dataset to COCO format using <a href=https://roboflow.com/formats/coco-json>Roboflow</a>. We used the same train, test, validation split that was used in Yolov5 model. 

We have train, test, validation folders. Each involves the images in jpg format and a json file for annotations. 

In [ ]:
import os 
os.listdir("/content/drive/MyDrive/Datasets/data_sign_coco")

['annotations', 'test_data', 'train_data', 'val_data', 'results']

In [ ]:
# read json file within train folder
import json
#read json files
open_file = open("/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json")
annotations_train = json.load(open_file)
open_file.close()

In [ ]:
print("annotation keys:", annotations_train.keys())
print("images example:", annotations_train["images"][0])
print("annotations example:", annotations_train["annotations"][0])
print("categories example:", annotations_train["categories"])

annotation keys: dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
images example: {'id': 0, 'license': 1, 'file_name': 'road376_png.rf.5a4e06e629169a0a8f300240e7a54d1d.jpg', 'height': 416, 'width': 416, 'date_captured': '2022-01-08T00:16:02+00:00'}
annotations example: {'id': 0, 'image_id': 0, 'category_id': 2, 'bbox': [54, 201, 134.5, 100], 'area': 13450, 'segmentation': [], 'iscrowd': 0}
categories example: [{'id': 0, 'name': 'road-signs', 'supercategory': 'none'}, {'id': 1, 'name': 'crosswalk', 'supercategory': 'road-signs'}, {'id': 2, 'name': 'speedlimit', 'supercategory': 'road-signs'}, {'id': 3, 'name': 'stop', 'supercategory': 'road-signs'}, {'id': 4, 'name': 'trafficlight', 'supercategory': 'road-signs'}]


## Modify Config
In the next step, we need to modify the config for the training. To accelerate the process, we finetune a detector using a pre-trained detector.

In [ ]:
os.getcwd()

'/content/mmdetection'

In [ ]:
os.listdir("./configs/mask_rcnn") # we will use mask_rcnn_r101_fpn_1x_coco.py

['mask_rcnn_x101_32x8d_fpn_1x_coco.py',
 'mask_rcnn_r50_fpn_poly_1x_coco.py',
 'mask_rcnn_r101_caffe_fpn_1x_coco.py',
 'mask_rcnn_x101_64x4d_fpn_mstrain-poly_3x_coco.py',
 'mask_rcnn_x101_32x4d_fpn_mstrain-poly_3x_coco.py',
 'README.md',
 'mask_rcnn_r50_fpn_2x_coco.py',
 'mask_rcnn_r50_caffe_fpn_1x_coco.py',
 'mask_rcnn_r50_caffe_fpn_mstrain_1x_coco.py',
 'mask_rcnn_x101_32x8d_fpn_mstrain-poly_1x_coco.py',
 'mask_rcnn_x101_32x8d_fpn_mstrain-poly_3x_coco.py',
 'mask_rcnn_x101_32x4d_fpn_2x_coco.py',
 'mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py',
 'mask_rcnn_r101_fpn_mstrain-poly_3x_coco.py',
 'mask_rcnn_r50_caffe_fpn_mstrain-poly_2x_coco.py',
 'mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py',
 'metafile.yml',
 'mask_rcnn_r50_caffe_c4_1x_coco.py',
 'mask_rcnn_r101_fpn_2x_coco.py',
 'mask_rcnn_x101_32x4d_fpn_1x_coco.py',
 'mask_rcnn_r50_fpn_fp16_1x_coco.py',
 'mask_rcnn_r50_caffe_fpn_poly_1x_coco_v1.py',
 'mask_rcnn_r101_caffe_fpn_mstrain-poly_3x_coco.py',
 'mask_rcnn_r50_fpn_mstrain

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r101_fpn_1x_coco.py')

In [ ]:
print(cfg.keys())
print(cfg.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix) 
print(cfg.data.test.ann_file) 
print(cfg.data.test.img_prefix)
print(cfg.model.roi_head.bbox_head.num_classes)
print(cfg.model.roi_head.mask_head.num_classes)

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow'])
data/coco/
data/coco/annotations/instances_train2017.json
data/coco/train2017/
data/coco/annotations/instances_val2017.json
data/coco/val2017/
data/coco/annotations/instances_val2017.json
data/coco/val2017/
80
80


In [ ]:
from mmdet.apis import set_random_seed

cfg["classes"] = ("road-signs", "crosswalk", "speedlimit", "stop", "trafficlight")
# define the data root
cfg.data_root = "/content/drive/MyDrive/Datasets/data_sign_coco/"
# define paths for train images and annotations
cfg.data.train.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json'
cfg.data.train.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/train_data/'
# define paths for validation images and annotations
cfg.data.val.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_val.json'
cfg.data.val.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/val_data/'
# define paths for test images and annotations
cfg.data.test.ann_file = '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_test.json'
cfg.data.test.img_prefix = '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/'

# change number of classes to 5 (default is 80)
cfg.model.roi_head.bbox_head.num_classes = 5
cfg.model.roi_head.mask_head.num_classes = 5

# load pre-trained model
cfg.load_from = 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r101_fpn_mstrain-poly_3x_coco/mask_rcnn_r101_fpn_mstrain-poly_3x_coco_20210524_200244-5675c317.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/Datasets/data_sign_coco/results'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


ModuleNotFoundError: ignored

In [ ]:
cfg.data

{'samples_per_gpu': 2,
 'test': {'ann_file': '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_test.json',
  'img_prefix': '/content/drive/MyDrive/Datasets/data_sign_coco/test_data/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'flip': False,
    'img_scale': (1333, 800),
    'transforms': [{'keep_ratio': True, 'type': 'Resize'},
     {'type': 'RandomFlip'},
     {'mean': [123.675, 116.28, 103.53],
      'std': [58.395, 57.12, 57.375],
      'to_rgb': True,
      'type': 'Normalize'},
     {'size_divisor': 32, 'type': 'Pad'},
     {'keys': ['img'], 'type': 'ImageToTensor'},
     {'keys': ['img'], 'type': 'Collect'}],
    'type': 'MultiScaleFlipAug'}],
  'type': 'CocoDataset'},
 'train': {'ann_file': '/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json',
  'img_prefix': '/content/drive/MyDrive/Datasets/data_sign_coco/train_data/',
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'LoadAnnotations', 'with_bbox': True, 'with_ma

In [ ]:
fname = './configs/mask_rcnn/customized_config.py'
with open(fname, 'w') as f:
    f.write(cfg)

TypeError: ignored

In [ ]:
print(cfg.keys())
print(cfg.data_root)
print(cfg.data.train.ann_file)
print(cfg.data.train.img_prefix)
print(cfg.data.val.ann_file)
print(cfg.data.val.img_prefix) 
print(cfg.data.test.ann_file)
print(cfg.data.test.img_prefix)
print(cfg.model.roi_head.bbox_head.num_classes)
print(cfg.model.roi_head.mask_head.num_classes)

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'classes', 'work_dir', 'seed', 'gpu_ids'])
/content/drive/MyDrive/Datasets/data_sign_coco/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_train.json
/content/drive/MyDrive/Datasets/data_sign_coco/train_data/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_val.json
/content/drive/MyDrive/Datasets/data_sign_coco/val_data/
/content/drive/MyDrive/Datasets/data_sign_coco/annotations/instances_test.json
/content/drive/MyDrive/Datasets/data_sign_coco/test_data/
5
5


In [ ]:
cfg.model.backbone.init_cfg

{'checkpoint': 'torchvision://resnet101', 'type': 'Pretrained'}

# Train new detector - with pre-trained model

In [ ]:
!python ./tools/train.py cfg.model

Traceback (most recent call last):
  File "./tools/train.py", line 196, in <module>
    main()
  File "./tools/train.py", line 94, in main
    cfg = Config.fromfile(args.config)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 335, in fromfile
    use_predefined_variables)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 181, in _file2dict
    check_file_exist(filename)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/path.py", line 23, in check_file_exist
    raise FileNotFoundError(msg_tmpl.format(filename))
FileNotFoundError: file "/content/mmdetection/cfg.model" does not exist


In [ ]:
cfg.keys()

dict_keys(['model', 'dataset_type', 'data_root', 'img_norm_cfg', 'train_pipeline', 'test_pipeline', 'data', 'evaluation', 'optimizer', 'optimizer_config', 'lr_config', 'runner', 'checkpoint_config', 'log_config', 'custom_hooks', 'dist_params', 'log_level', 'load_from', 'resume_from', 'workflow', 'classes', 'work_dir', 'seed', 'gpu_ids'])

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import mmcv
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model)


# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-01-15 17:13:06,519 - mmdet - INFO - load checkpoint from local path: checkpoints/mask_rcnn_r101_fpn_1x_coco_20200205-d4b0c5d6.pth


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


FileNotFoundError: ignored

In [ ]:
cfg.model.test_cfg

{'rcnn': {'mask_thr_binary': 0.5,
  'max_per_img': 100,
  'nms': {'iou_threshold': 0.5, 'type': 'nms'},
  'score_thr': 0.05},
 'rpn': {'max_per_img': 1000,
  'min_bbox_size': 0,
  'nms': {'iou_threshold': 0.7, 'type': 'nms'},
  'nms_pre': 1000}}